<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out = ''.join([booster_version for i, booster_version in enumerate(
        table_cells.strings) if i % 2 == 0][0:-1])
    return out


def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out = [i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not (colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
response = requests.get(static_url, headers=headers)

In [6]:
response.status_code

200

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one

In [11]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Obtain list of beautiful soup tag objects
find_th = first_launch_table.find_all('th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Process one tag element at a time, returning none or the column name
for t in find_th:
    name = extract_column_from_header(t)

# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Filter out the 'none's and names with no length
    if (name is not None) and (len(name) > 0):
        column_names.append(name)

Check the extracted column names


In [12]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [13]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
# Initialise counter
extracted_row = 0

# List to store all launches
launch_data = []

# Loop over SpaceX tables
for table_number, table in enumerate(
    soup.find_all('table', "wikitable plainrowheaders collapsible")
):
    for rows in table.find_all("tr"):

        # ---- Identify valid launch rows via flight number ----
        flag = False
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()

        # Extract table cells
        row = rows.find_all('td')

        # ---- Process only valid launch rows ----
        if flag:
            extracted_row += 1
            launch_dict = {}

            # ---------- 1) GET ROW INFO ----------
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]

            bv = booster_version(row[1])
            if not bv:
                bv = row[1].get_text(strip=True)

            launch_site = row[2].get_text(strip=True)
            payload = row[3].get_text(strip=True)
            payload_mass = get_mass(row[4])
            orbit = row[5].get_text(strip=True)
            customer = row[6].get_text(strip=True)
            launch_outcome = list(row[7].strings)[0]
            booster_landing = landing_status(row[8])

            # ---------- 2) PUT EVERYTHING INTO launch_dict ----------
            launch_dict['Flight No.'] = int(flight_number)
            launch_dict['Date'] = date
            launch_dict['Time'] = time
            launch_dict['Version Booster'] = bv
            launch_dict['Launch site'] = launch_site
            launch_dict['Payload'] = payload
            launch_dict['Payload mass'] = payload_mass
            launch_dict['Orbit'] = orbit
            launch_dict['Customer'] = customer
            launch_dict['Launch outcome'] = launch_outcome
            launch_dict['Booster landing'] = booster_landing

            # ---------- 3) PRINT ----------
            print(launch_dict)

            # Store result
            launch_data.append(launch_dict)

{'Flight No.': 1, 'Date': '4 June 2010', 'Time': '18:45', 'Version Booster': 'F9 v1.07B0003.18', 'Launch site': 'CCAFS,SLC-40', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch outcome': 'Success\n', 'Booster landing': 'Failure'}
{'Flight No.': 2, 'Date': '8 December 2010', 'Time': '15:43', 'Version Booster': 'F9 v1.07B0004.18', 'Launch site': 'CCAFS,SLC-40', 'Payload': 'Dragondemo flight C1(Dragon C101)', 'Payload mass': 0, 'Orbit': 'LEO(ISS)', 'Customer': 'NASA(COTS)NRO', 'Launch outcome': 'Success', 'Booster landing': 'Failure'}
{'Flight No.': 3, 'Date': '22 May 2012', 'Time': '07:44', 'Version Booster': 'F9 v1.07B0005.18', 'Launch site': 'CCAFS,SLC-40', 'Payload': 'Dragondemo flight C2+[18](Dragon C102)', 'Payload mass': '525 kg', 'Orbit': 'LEO(ISS)', 'Customer': 'NASA(COTS)', 'Launch outcome': 'Success', 'Booster landing': 'No\xa0attempt\n'}
{'Flight No.': 4, 'Date': '8 October 2012', 'Time': '00:35', 'Version Boos

In [ ]:
'''
# Initialise counter
extracted_row = 0

# Extract each table - Loop over SpaceX tables
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        # check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # get table element, extract table cells
        row = rows.find_all('td')
        # Only process valid rows; if flag is a number, save cells in a dictonary
        if flag:
            launch_dict = {}
            extracted_row += 1

            # FLIGHT NUMBER
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'] = int(flight_number)
            # print(flight_number)
            
            # DATE 
            datatimelist = date_time(row[0])
            # Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date
            # print(date)

            # TIME
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'] = time
            # print(time)

            # BOOSTER VERSION
            # Append the bv into launch_dict with key `Version Booster`
            bv = booster_version(row[1])
            if not (bv):
                bv = row[1].a.string
            #print(bv)
            launch_dict['Version Booster'] = bv

            # LAUNCH SITE
            # Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            # print(launch_site)
            launch_dict['Launch site'] = launch_site

            # PAYLOAD
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            # print(payload)
            launch_dict['Payload'] = payload

            # PAYLOAD MASS
            # Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            # print(payload)
            launch_dict['Payload mass'] = payload_mass

            # ORBIT
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            # print(orbit)
            launch_dict['Orbit'] = orbit

            # CUSTOMER
            # Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            # print(customer)
            launch_dict['Customer'] = customer

            # LAUNCH OUTCOME
            # Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            # print(launch_outcome)
            launch_dict['Launch outcome'] = launch_outcome

            # BOOSTER LANDING
            # Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            # print(booster_landing)
            launch_dict['Booster landing'] = booster_landing
'''

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [27]:
df = pd.DataFrame(launch_data)

In [28]:
df.describe(include='all')

,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
count,121.000000,121,121,121,121,121,121,121,121,121,121
unique,NaN,121,116,56,10,121,84,13,54,3,9
top,NaN,4 June 2010,04:45,F9 B5,"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,"15,600 kg",LEO,SpaceX,Success\n,Success
freq,NaN,1,3,19,39,1,24,38,26,88,80
mean,61.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,35.073732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,31.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,61.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,91.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [29]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
